In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from collections import defaultdict

In [ ]:
WS_REAL = defaultdict(list)
valid_final_season = {}

FEAT = ['Age','WS']

for YR in range(1980,2020):
    table = np.load('tables_{}.pkl'.format(YR))
    for team in table:
        stats = table[team]['advanced'][FEAT]
        for item in stats.itertuples():
            try:
                v = [0.0 if _=='' else _ for _ in item[1:]]
                #hgt = table[team]['roster'].loc[item[0]]['Ht']
                #hgt = [int(_) for _ in hgt.split('-')]
                #hgt = 12*hgt[0] + hgt[1]
                #v.append(hgt)
                WS_REAL[item[0]].append(np.array(v).astype(np.float))
            except:
                print(item)

In [ ]:
d = []
for k in WS_REAL:
    t = np.array(WS_REAL[k])
    tm = t[:,2]#*t[:,3]
    tm = tm.astype(np.float)
    d.append(np.vstack([[k for _ in range(tm.shape[0])],tm,t[:,1]]).T)

In [ ]:
dft = pd.DataFrame(np.vstack(d),columns=['name','mp','ast'])

In [ ]:
LIM = 200
d2 = dft[(dft.ast.astype(np.float) == 0.0) & (dft.mp.astype(np.float) > LIM)]

a,b = d2.shape[0],dft[(dft.mp.astype(np.float) > LIM)].shape[0]
print(100*a/b,a,b)
d2

In [ ]:
table = np.load('tables_{}.pkl'.format(2019))
for team in table:
    stats = table[team]['advanced']
    for item in stats.itertuples():
        valid_final_season[item[0]] = 1


In [ ]:
X = []
y = []
for name in WS_REAL:
    stats = WS_REAL[name]
    yrs = len(stats)
    X += stats
    for i in range(yrs-1):
        y.append(0.0)
    y.append(float(name not in valid_final_season))

In [ ]:
from sklearn.linear_model import LogisticRegression, ElasticNet, Ridge,RidgeClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.preprocessing import PolynomialFeatures,StandardScaler

In [ ]:
Xo = np.array(X)
yo = np.array(y)


In [ ]:
X = Xo.copy()
y = yo.copy()
X[:,0] -= 18
X[:,0] *= X[:,0]
#X[:,-1] /= X[:,-1].max()
#X = np.hstack([X,(X[:,0] * X[:,1]).reshape((-1,1))])

In [ ]:
fexp = PolynomialFeatures(2,interaction_only=True)
scal = StandardScaler()
pX = X#fexp.fit_transform(X)#fexp.fit_transform(X)
#pX[:,0] = pX[:,0]**2

clf = LogisticRegression(C=1e6,solver='liblinear',class_weight='balanced')
clf.fit(pX,y)
clf.score(pX,y),(1-y.sum()/y.shape[0])

In [ ]:
_ = plt.hist(clf.predict_proba(pX)[:,1],50,density=True)

In [ ]:
FEAT,1/clf.coef_,clf.intercept_

In [ ]:
import statsmodels.api as sm

In [ ]:
X2 = sm.add_constant(pX)
logit_mod = sm.Logit(y,pd.DataFrame(X2,columns=['1'] + FEAT ))#fexp.get_feature_names(FEAT)))
logit_res = logit_mod.fit(disp=0)
print(logit_res.summary())
np.mean((logit_res.predict() >0.5)== y.astype(np.bool))

In [ ]:
xp = np.linspace(18,50)
p = lambda x: 1.0/(1.0+np.exp(-x))
for WS in [-1,0,1,3,5,7]:
    #plt.plot(xp,p(clf.intercept_ + clf.coef_[0,0]*xp),label='WS0')
    plt.plot(xp,p(clf.intercept_ + clf.coef_[0,0]*(xp-18)**2 + clf.coef_[0,1]*WS   ),label='WS {}'.format(WS))
plt.xlim(18,55)
plt.legend()
plt.xlabel('Age')

plt.ylabel('Prob of Retiring')
plt.title('NBA (Age-18)^2 + WS Model')
plt.ylim(0,1)
plt.tight_layout()

plt.savefig('retire.png',facecolor='w',edgecolor='w')

In [ ]:
clf.coef_[0,1]/clf.coef_[0,-1]

In [ ]:
pX

In [ ]:
df2 = pd.read_csv('beta_stats.csv')#big_stat


In [ ]:
from matplotlib.colors import LogNorm
dft = df2[df2.MP > 8]
_ = plt.hexbin(dft.Ovr,dft['PER'],gridsize=40,norm=LogNorm())
#plt.ylim(-5,40)
#plt.xlim(30,85)
plt.xlabel('Ovr')
plt.ylabel('PER')
xp = np.linspace(30,90)
#plt.plot(xp,(1/64)*np.maximum(xp-40,0)**2,c='r')
plt.title('bbgm ovr -> PER')
plt.savefig('wsovr3.png',edgecolor='w',facecolor='w')

In [ ]:
WS_FAKE = defaultdict(list)
valid_final_season2 = {}

FEAT = ['Name','Age','WS','TS%','3PAr','Pot']

for item in df2[FEAT].itertuples():
    try:
        v = [0.0 if _=='' else _ for _ in item[2:]]
        #hgt = table[team]['roster'].loc[item[0]]['Ht']
        #hgt = [int(_) for _ in hgt.split('-')]
        #hgt = 12*hgt[0] + hgt[1]
        #v.append(hgt)
        WS_FAKE[item[1]].append(np.array(v).astype(np.float))
    except:
        raise#print(item)

In [ ]:
WS_FAKE['Doug Moten']

In [ ]:
X2 = []
y2 = []
for name in WS_FAKE:
    stats = WS_FAKE[name]
    yrs = len(stats)
    X2 += stats
    for i in range(yrs-1):
        y2.append(0.0)
    y2.append(1.0)

In [ ]:
X2 = np.array(X2)
y2 = np.array(y2)


In [ ]:

clf2 = LogisticRegression(C=1e6,solver='liblinear',class_weight='balanced')
clf2.fit(X2,y2)
clf2.score(X2,y2),(1-y2.sum()/y2.shape[0])

In [ ]:
_ = plt.hist(clf2.predict_proba(X2)[:,1],50,density=True)

In [ ]:
FEAT,clf2.coef_,clf2.intercept_

In [ ]:
X3 = sm.add_constant(X2)
logit_mod = sm.Logit(y2,X3)
logit_res = logit_mod.fit(disp=0)
print(logit_res.summary())
np.mean((logit_res.predict() >0.5)== y2.astype(np.bool))

In [ ]:
xp = np.linspace(20,40)
p = lambda x: 1.0/(1.0+np.exp(-x))
for WS in [-1,0,1,3,5,7]:
    #plt.plot(xp,p(clf.intercept_ + clf.coef_[0,0]*xp),label='WS0')
    plt.plot(xp,p(clf2.intercept_ + clf2.coef_[0,0]*xp + clf2.coef_[0,1]*WS+ clf2.coef_[0,2]*30+ clf2.coef_[0,3]*5 + clf2.coef_[0,4]*50  ),label='WS {}'.format(WS))
plt.xlim(20,46)
plt.legend()
plt.title('BBGM, TS%:30, 3PAr:5')
plt.xlabel('Age')
plt.ylabel('P(Retire | Age, WS)')
plt.tight_layout()
plt.savefig('retireFAKE.png',facecolor='w',edgecolor='w')

In [ ]:
clf3 = ElasticNet(alpha=0)
df3 = df2[df2.Age > 25]
clf3.fit(np.array(df3.Ovr).reshape((-1,1))**2,df3.WS)
clf3.score(np.array(df3.Ovr).reshape((-1,1)),df3.WS)

In [ ]:
clf3.coef_,clf3.intercept_

In [ ]:
xp = np.linspace(20,60)
p = lambda x: 1.0/(1.0+np.exp(-x))
for POT in [40,50,60,70]:
    #plt.plot(xp,p(clf.intercept_ + clf.coef_[0,0]*xp),label='WS0')
    plt.plot(xp,p(-2.5 + xp**2 * 0.006 - 1* ((POT-40)/4) ),label='Pot {}'.format(POT))
plt.xlim(20,55)
plt.ylim(0,1)
plt.legend()
plt.xlabel('Age')

plt.ylabel('P(Retire | Age, Pot)')
plt.title('Fake model')

plt.tight_layout()

plt.savefig('retire2.png',facecolor='w',edgecolor='w')

In [ ]:
clf.intercept_ ,clf.coef_[0,0],clf.coef_[0,1]

In [ ]:
dft = df2[(df2.Ovr < 62) & (df2.WS > 16)]
dft

In [ ]:
pd.set_option("display.precision", 2)
df2[(df2.Season == 2187) & (df2.Team == 'MIA')].sort_values('MP',0,False)[['Name','Pos','Age','Ovr','Salary','GS','MP','WS','PTS','TRB','AST','STL','Blk','TOV']]

In [ ]:
from tabulate import tabulate

In [ ]:
print(tabulate(df2[(df2.Season == 2187) & (df2.Team == 'MIA')].sort_values('MP',0,False)[['Name','Pos','Age','Salary','GS','MP','WS','PTS','TRB','AST','STL','Blk','TOV']]))